In [1]:
class motion_detection:
    
    # to-do : un load model
    
    # voir pour mettre les parametre variate dans la fonction start plutot que dans l'init

    def _create_model(self):
        # creation of a sequential model
        # 3 layers : 2 LSTM and 1 Dense
        # LSTM = Long Short Memory layer
        # Dense = a regular densely-connected NN layer
    
        model = Sequential()
        # no parameter of Sequential function
        model.add(LSTM( self.nb_neurons[0], activation = self.activation, 
                       return_sequences = True,
                       input_shape = self.input_shape,
                       recurrent_dropout = self.recu_dropout))
        model.add(LSTM(self.nb_neurons[1],
                       recurrent_dropout = self.recu_dropout))
        #model.add(BatchNormalization())
        #model.add(Dropout(self.recu_dropout))
        #model.add(BatchNormalization())
        model.add(Dense( self.nb_labels, activation = 'softmax' ))
        
        model.compile( loss = "mse", 
                      optimizer = optimizers.Adam(lr = self.learning_rate),
                     metrics = ['accuracy'])
        
#         log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#         tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        print(model.summary())
        
        return model
    
    def _create_model_hirerachique(self):
        # fonctionne uniquement sur les données de la neuron pro
        
        model = Sequentiel()
        
        tronc = Sequentiel()
        # vertebre : 43 - 48
        # vertebre 1 : 49 - 54
        # vertebre 2 : 55 - 60
        # vertebre 3 : 61 - 66
        # nuque : 67 - 72
        # tete : 73 - 78
        
        
        bras_g = Sequentiel()
        # epaule gauche : 217 - 222
        # biceps gauche : 223 - 228
        # avant-bras gauche : 229 - 234
        # main gauche : 235 - 240
        
        bras_d = Sequentiel()
        # epaule droite : 79 - 84
        # biceps droit : 85 - 90
        # avant-bras droit : 91 - 96
        # main droite : 97 - 102
        
        jambe_g = Sequentiel()
        # cuisse gauche : 25 - 30
        # mollet gauche : 31 - 36
        # pied gauche : 37 - 42
        
        jambe_d = Sequentiel()
        # cuisse droite : 7 - 12
        # mollet droit : 13 - 18
        # pied droit : 19 - 24
        
        return model
    
    def _model_fit(self):
        # training time
        start_time = time.time()
        self.algo_start_time = start_time
        self.history = self.model.fit(self.x_train, self.y_train , 
                       epochs=self.nb_epochs, 
                       batch_size=self.batch_size_train, 
                       verbose=self.nb_verbose,
                        validation_split=0.3,
                        shuffle=True)
        end_time = time.time()
        total_time = end_time - start_time
        self.time_train = total_time
        if self.nb_verbose == 1 | self.nb_verbose == 2:
            print("Training time :", total_time)
        
        plt.plot(self.history.history['accuracy'])
        plt.plot(self.history.history['loss'])
        
    def _model_prediction(self):
        # prediction time
        start_time = time.time()
        prediction = self.model.predict(self.x_test)
        end_time = time.time()
        total_time = end_time - start_time
        if self.nb_verbose == 1 | self.nb_verbose == 2:
            print("Prediction time :", total_time)
        self.time_predict = total_time
        self.prediction = prediction
    
    def _model_evaluate(self):
        results = self.model.evaluate(self.x_test, 
                                      self.y_test, 
                                      batch_size=self.batch_size_test, 
                                      verbose=self.nb_verbose)
        print('test loss, test acc:', results)
        self.results = results
        self.get_confusion_matrix()

    def start(self,dataset, x_train, y_train, 
                 x_test, y_test, nb_neurons, learning_rate, 
                 nb_epochs, batch_size_train, recu_dropout,
                 activation, 
                 batch_size_test, nb_verbose):
        
        self.dataset = dataset
        
        self.x_train = np.array(x_train)
        self.y_train = np.array(y_train)
        self.x_test = x_test
        self.y_test = y_test
        
        self.learning_rate = learning_rate
        self.nb_neurons = nb_neurons
        self.input_shape = x_train[0].shape
        self.nb_labels = 2
        self.nb_epochs = nb_epochs
        self.batch_size_train = batch_size_train
        self.recu_dropout = recu_dropout
        self.activation = activation
        self.batch_size_test = batch_size_test
        self.nb_verbose = nb_verbose
        
        self.model = self._create_model()
        self.results = np.array([0,0], dtype="float")
        self.predictions = np.array([])
        
        self.time_train = 0
        self.time_predict = 0
        
        self.algo_start_time = 0
        if self.nb_verbose == 1 | self.nb_verbose == 2:
            print("Model training")
        self._model_fit()
        if self.nb_verbose == 1 | self.nb_verbose == 2:
            print("Model prediction")
        self._model_prediction()
        if self.nb_verbose == 1 | self.nb_verbose == 2:
            print("Model evaluation")
        self._model_evaluate()
        if self.nb_verbose == 1 | self.nb_verbose == 2:
            print("Model backup")
        self._model_save()
        if self.nb_verbose == 1 | self.nb_verbose == 2:
            print("End")
        
    def _model_save(self):
        # Save only when it's needed
        path = 'model'
        if os.path.isdir(path) == False:
            try:
                os.mkdir(path)
            except OSError:
                print ("Creation of the directory %s failed" % path)
            else:
                print ("Successfully created the directory %s " % path)
                
        nb_saved_model = self._count_files_dir() + 1
        #print(nb_saved_model)
        name_model = 'model_'+str(nb_saved_model)
        
        self.model.save(path+'/'+name_model)
        log = open(path+'/log.txt','a')
        
        log.write('\n'+name_model+'|'+self._to_string())
        log.close()
        
    def _to_string(self):
        tmp = str('Dataset:'+self.dataset+
                   '|neurons:'+str(self.nb_neurons)+
                   '|epochs:'+str(self.nb_epochs) + 
                   '|batch_size:'+str(self.batch_size_train) + 
                   '|activation:'+self.activation + 
                   '|batch_size(test):'+str(self.batch_size_test) + 
                   '|start_time:'+str(self.algo_start_time) + 
                   '|time_train:'+ str(self.time_train) + 
                   '|time_predict:'+str(self.time_predict) + 
                   '|['+str(self.results[0])+','+str(self.results[1])+']')
        return tmp
        
    def _count_files_dir(self):
        count = 0
        for path in pathlib.Path("model/").iterdir():
            if path.is_file():
                count += 1

        return count
    
    def get_confusion_matrix(self):
        print(confusion_matrix(y_test.argmax(axis=1), md.prediction.argmax(axis=1)))
        
     